In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from tensorflow.python.keras import models
from tensorflow.python.keras import layers

2023-06-07 17:16:48.365060: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [37]:
# https://www.kaggle.com/datasets/christophercorrea/prisoners-and-crime-in-united-states
all_deaths = pd.read_csv("datasets/all_deaths.csv", encoding='latin-1')
all_jails = pd.read_csv("datasets/all_jails.csv", encoding='latin-1')


# https://bjs.ojp.gov/library/publications/prisoners-2021-statistical-tables#:~:text=The%20U.S.%20prison%20population%20was,decrease%20from%202011%20(1%2C599%2C000)
p21stt01 = pd.read_csv("datasets/p21stt01.csv", encoding='latin-1')
prisonersPerYear = pd.read_csv("datasets/p21stat02.csv", encoding='latin-1')


In [38]:
# clean prisoner data


prisonersPerYear = prisonersPerYear.drop(range(0, 10)).reset_index(drop=True)
prisonersPerYear.columns = ["Year", "Sentenced Prisoners"]

prisonersPerYear



,Year,Sentenced Prisoners
0,1996,"1,137,722"
1,1997,"1,194,334"
2,1998,"1,256,474"
3,1999,"1,304,081"
4,2000,"1,334,174"
5,2001,"1,345,217"
6,2002,"1,380,516"
7,2003,"1,408,361"
8,2004,"1,433,728"
9,2005,"1,462,866"
